In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
## Camera

import cv2

# Load the pre-trained Haar cascade classifier for car detection
car_cascade = cv2.CascadeClassifier('haarcascade_car.xml')

# Function to detect cars in an image
def detect_cars(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cars = car_cascade.detectMultiScale(gray, 1.1, 3)
    for (x, y, w, h) in cars:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
    return image

# Main function
def main():
    # Capture video from webcam
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect cars in the frame
        frame_with_cars = detect_cars(frame)

        # Display the frame with detected cars
        cv2.imshow('Parking Management System', frame_with_cars)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [3]:
net = cv2.dnn.readNet("yolov3 (1).weights", "yolov3.cfg")


In [19]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3 (1).weights", "yolov3.cfg")
classes = []
with open("coco (2).names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get output layer names
output_layer_names = net.getUnconnectedOutLayersNames()

# Function to detect cars in an image using YOLO
def detect_cars(image):
    height, width, _ = image.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layer_names)

    # Showing informations on the screen
    cars_detected = 0
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4 and class_id == 2:  # 2 corresponds to "car" class in COCO dataset
                # Object detected is a car with confidence > 0.5
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cars_detected += 1

    return image, cars_detected

# Function to count parking spots
def count_parking_spots(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range of green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    # Threshold the HSV image to get only green colors
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    # Setup SimpleBlobDetector parameters
    params = cv2.SimpleBlobDetector_Params()

    # Filter by color
    params.filterByColor = True
    params.blobColor = 255

    # Create a detector with the parameters
    detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs
    keypoints = detector.detect(mask_green)

    # Draw detected blobs as green circles
    im_with_keypoints = cv2.drawKeypoints(image, keypoints, np.array([]), (0, 255, 0),
                                          cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # Count parking spots
    parking_spots = len(keypoints)

    return parking_spots, im_with_keypoints

# Main function
def main():
    # Path to the video file
    video_path = 'Parking Spotter -- AI based Video Analytics.mp4'
    # Capture video from file
    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Detect cars in the frame using YOLO
        frame_with_cars, _ = detect_cars(frame)

        # Count parking spots
        num_parking_spots, frame_with_spots = count_parking_spots(frame_with_cars)

        # Calculate available parking spots
        num_cars = _
        num_available_spots = num_parking_spots - num_cars
        if num_available_spots < 0:
            num_available_spots = 0

        # Display the frame with detected cars and parking availability
        cv2.putText(frame_with_spots, f'Cars detected: {num_cars}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame_with_spots, f'Available spots: {num_available_spots}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Parking Management System', frame_with_spots)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
